In [1]:
import torch
from model import make_model
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from utils import greedy_decode

I0909 15:11:14.576323 140734738472384 file_utils.py:39] PyTorch version 1.6.0 available.


In [34]:
num_decoder_layers = 3
model_path = './chatbot-ckpt/step_60000.pth'

In [35]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model, vocab = make_model(num_decoder_layers)
model.load_state_dict(torch.load(model_path, map_location=device)['model'])
model.eval()
sp  = SentencepieceTokenizer(get_tokenizer())

I0909 14:30:43.259646 140734987816384 configuration_utils.py:300] Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 8002
}

using cached model
using cached model
using cached model


In [76]:
question = '사랑했던 그 사람의 다른 모습이 보인거예요??'
max_len = 20

tokens = sp(question)
tokens = ['[CLS]'] + tokens + ['[SEP]']
token_ids = [vocab.token_to_idx[tok] for tok in tokens]
# unsqueeze(0) for Batch position (zzingae)
token_ids = torch.tensor(token_ids).unsqueeze(0)
# attention score: [Batch, Head, tgt_length, src_length] in src_attn (zzingae)
# unsqueeze(1) for tgt_length position (zzingae)
attn_mask = (token_ids != vocab.token_to_idx['[PAD]']).unsqueeze(1).long()

answer = greedy_decode(model, token_ids, attn_mask, max_len, vocab)

print('question: {}'.format(sp(question)))
print('answer: '+''.join([vocab.idx_to_token[idx] for idx in answer[0,1:]]))

question: ['▁사랑', '했던', '▁그', '▁사람의', '▁다른', '▁모습이', '▁보', '인', '거', '예', '요', '??']
answer: ▁마음을▁이어보고▁싶어하는▁사람이▁만들었어요.
